In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell import Cell
from openpyxl.styles import Color, Font, PatternFill

In [2]:
rd = 20200331
# Insert the directory where to save the file
# The Reporting Date will be requested and saved in a global var
def get_rd(directory):
    os.chdir(directory)
    global rd
    rd = str(input("Please enter the RD (YYYYMMDD): "))
    
cno_user = ""
cno_pwd = ""
# Input the password when it's asked
# It will be used later for the connection
def get_pwd():
    # Getting user from env
    global cno_user
    cno_user = "CNO_" + getpass.getuser()

    global cno_pwd
    # Prompting for pwd
    print("Please, enter the password for your user: ", cno_user)
    cno_pwd = getpass.getpass()
    if(cno_pwd == ""):
        print("Error - password entered as blank")
    else:
        print("Password entered")



In [3]:
# Insert the workspace in the parameter
# The connection will be made
def connect_to_DB(ws):     
    
    print("Connection successfully")

In [4]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/Consolidated Exposure"
get_rd(directory)
get_pwd()


Please enter the RD (YYYYMMDD): 20200331
Please, enter the password for your user:  CNO_WZHARBC
········
Password entered


In [5]:
connect_to_DB(46)

Connection successfully


In [6]:
# Insert the parameters required
# File_name -> the name of the file you want to save
# Sheet_name -> the name of the sheets that will be created
# Ws_nr -> the number of the respective WS that the connection will be made with
# Sqls -> the sqls that will be run
def save_file(file_name, years, quarters):
#     # Create file
#     wb = Workbook()
#     for year in years:
#         for quarter in quarters:
#             name = str(year) + str(quarter)
#             wb.create_sheet(name)
        
# #     std = wb.get_sheet_by_name('Sheet')
# #     wb.remove_sheet(std)
#     wb.save(filename = file_name)
    
#     dfs = []
    
    for year in years:
        for quarter in quarters:
            try:
                temp = year + quarter
                
                logon_string_RAY = str("oracle+cx_oracle://" + cno_user + ":" + cno_pwd + "@localhost:42521/PREP000")
                connection_RAY = create_engine(logon_string_RAY)
                connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                        reporting_date = temp, partition = 46))
                sql = """select partition_key, reporting_date, booking_segment, booking_company, risk_country, rating_model, internal_rating,	industry_code_1, industry_code_2, tenor_residual,	residual_maturity, currency, sum(ead_pre_ccf) as ead_pre_ccf, sum(ead) as ead, sum(rwa) as rwa
from  rzb_cdr		
group by partition_key, reporting_date, booking_segment, booking_company, risk_country, rating_model, internal_rating,	industry_code_1,  industry_code_2,  tenor_residual, residual_maturity, currency	
                """
                df_temp= pd.read_sql_query( sql, connection_RAY)
                df_temp.to_excel(year + quarter + "_NRT.xlsx")  
#                 dfs.append(df_temp)
            except:
#                 dfs.append(pd.DataFrame())
                print("WS is not created")
    
#     # Select SQLs
#     for index, sheet in enumerate(sheet_name):
#         try:
#             connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
#                             reporting_date = rd, partition = ws_nr[index]))
#             df_temp= pd.read_sql_query( sqls[index], connection_RAY)
#             dfs.append(df_temp)
#         except:
#             dfs.append(pd.DataFrame())
#             print( str(ws_nr[index]) + " is not created")
            
#     writer = pd.ExcelWriter(file_name)
#     for index, df in enumerate(dfs):
#         try:
#             df.to_excel(writer, index=False, sheet_name = sheet_name[index])
#         except:
#             print()
    
#     writer.save()
#     print('DataFrame is written to Excel File successfully.')

In [7]:
file_name = "Retail.xlsx"
years = ["2020", "2019", "2018", "2017", "2016", "2015", "2014", "2013", "2012", "2011"]
quarters = ["0331", "0630", "0930", "1231"]

save_file(file_name, years, quarters)

In [63]:
years = ["2021"]
quarters = ["0630", "0930", "1231"]

save_file(file_name, years, quarters)